In [1]:
import getpass
import os
import keras

import logging
## Set default log level to WARNING, set this to INFO if you want more messages
logging.basicConfig(level=logging.WARNING)

SM_DIR='/users/6/croal008/5527/ddc'
DATASET='fraxtil'

# experiment dir under ddc/experiments/<your_x500>
EXP_DIR=os.path.join(SM_DIR, 'experiments', getpass.getuser())

if not os.path.exists(EXP_DIR):
    print("Making experiment directory: " + EXP_DIR)
    os.mkdir(EXP_DIR)

2025-05-13 20:41:28.070446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-13 20:41:28.085659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-13 20:41:28.090097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 20:41:28.102080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 20:41:30.430591: W tensorflow/compiler/tf2

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4903407901177197181
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 45716668416
locality {
  bus_id: 4
  numa_node: 3
  links {
  }
}
incarnation: 17859747109337276203
physical_device_desc: "device: 0, name: NVIDIA A40, pci bus id: 0000:08:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
Num GPUs Available:  1


2025-05-13 20:41:34.260055: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /device:GPU:0 with 43598 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:08:00.0, compute capability: 8.6


### testing... 👽

In [5]:
training_args_dict = {
    'train_txt_fp': f'{SM_DIR}/data/chart_onset/{DATASET}/mel80hop441/{DATASET}_train.txt',
    'valid_txt_fp': f'{SM_DIR}/data/chart_onset/{DATASET}/mel80hop441/{DATASET}_valid.txt',
    'test_txt_fp': f'{SM_DIR}/data/chart_onset/{DATASET}/mel80hop441/{DATASET}_test.txt',
    'experiment_dir': EXP_DIR,
    'feat_diff_coarse_to_id_fp': f'{SM_DIR}/labels/{DATASET}/diff_coarse_to_id.txt',
    
    # Audio features
    'audio_context_radius': 7,
    'audio_nbands': 80,
    'audio_nchannels': 3,
    'audio_select_channels': '0,1,2',
    
    # CNN parameters
    'cnn_filter_shapes': '3,3,16,3,3,32,3,3,64,3,3,128',
    'cnn_pool': '1,2,1,2,1,2',
    'use_residual_connections': True,
    
    # RNN parameters (not using)
    'rnn_cell_type': 'lstm',
    'rnn_size': 200,
    'rnn_nlayers': 0, # not using rnn
    'rnn_nunroll': 1,
    'rnn_keep_prob': 0.5,
    
    # Transformer parameters
    'd_model': 256,
    'num_transformer_layers': 4,
    'num_heads': 8,
    'ff_dim': 512,
    'do_transformer': True,
    
    # Multi-task learning parameters
    'enable_multi_task': True,  # Enable multi-task learning
    'task_weights': '0.7,0.3',  # 70% weight for onset detection, 30% for density prediction
    
    # Dense layer parameters
    'dnn_nonlin': 'relu',
    'dnn_sizes': '256,128',
    'dnn_keep_prob': 0.5,
    
    # Training parameters
    'batch_size': 256,
    'norm_type': 'batch',
    'weight_strategy': 'rect',
    'exclude_onset_neighbors': 2,
    'grad_clip': 5.0,
    'opt': 'adam',
    'lr': 0.001,
    'lr_decay_rate': 0.95,
    'lr_decay_delay': 5,
    'nbatches_per_ckpt': 10000,
    'nbatches_per_eval': 10000,
    'nepochs': 20,
    
    # Evaluation parameters
    'eval_window_type': 'hamming',
    'eval_window_width': 5,
    'eval_align_tolerance': 2,
    'z_score': True,
    'exclude_pre_onsets': True,
    'exclude_post_onsets': True,
}


from onset_train_multitask import train_onset_model, parse_args
training_args = parse_args(get_defaults=True)
vars(training_args).update(training_args_dict)

from onset_net_multitask import OnsetNet
train_onset_model(training_args, OnsetModel=OnsetNet)

Loading data
Selecting channels [0, 1, 2] from data
Loading validation metrics
Flattening datasets into charts
Train set: 256 charts, valid set: 32 charts, test set: 32 charts
Feature configuration (nfeats=5): {'time_context_radius': 7, 'diff_feet_to_id': None, 'diff_coarse_to_id': {'Beginner': 0, 'Easy': 1, 'Medium': 2, 'Hard': 3, 'Challenge': 4}, 'diff_dipstick': False, 'freetext_to_id': None, 'beat_phase': False, 'beat_phase_cos': False}
Exclusions: {'randomize_charts': False, 'exclude_onset_neighbors': 2, 'exclude_pre_onsets': True, 'exclude_post_onsets': True, 'include_onsets': True}
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


Model configuration: {'audio_context_radius': 7, 'audio_nbands': 80, 'audio_nchannels': 3, 'nfeats': 5, 'cnn_filter_shapes': [(3, 3, 16), (3, 3, 32), (3, 3, 64), (3, 3, 128)], 'cnn_init': <tensorflow.python.ops.init_ops.UniformUnitScaling object at 0x7fb050d92210>, 'cnn_pool': [(1, 2), (1, 2), (1, 2)], 'cnn_rnn_zack': False, 'zack_hack': 0, 'rnn_cell_type': 'lstm', 'rnn_size': 200, 'rnn_nlayers': 0, 'rnn_init': <tensorflow.python.ops.init_ops.RandomUniform object at 0x7fb090641a10>, 'rnn_nunroll': 1, 'rnn_keep_prob': 0.5, 'dnn_sizes': [256, 128], 'dnn_init': <tensorflow.python.ops.init_ops.UniformUnitScaling object at 0x7fb053592b90>, 'dnn_keep_prob': 0.5, 'dnn_nonlin': 'relu', 'pooling_method': 'max', 'grad_clip': 5.0, 'opt': 'adam', 'norm_type': 'batch', 'd_model': 256, 'num_transformer_layers': 4, 'num_heads': 8, 'ff_dim': 512}
Creating train model
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2025-05-13 20:41:37.341666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43598 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:08:00.0, compute capability: 8.6
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Done evaluating
Saving model weights...
Done saving!
Evaluating...


/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527

Done evaluating
Completed epoch 20
Epoch mean cross-entropy (nats) 0.18602925539016724
Evaluating...
Calculating perchart and micro thresholds for validation data
Diff perchart thresholds: {'Challenge': 0.5, 'Hard': 0.25127184}
All perchart threshold: 0.37563592195510864
Diff_micro thresholds: {'Challenge': 1.0, 'Hard': 1.2894388e-06}
All micro thresholds: 1.0


/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527 Project/ddc/learn/josh/onset_train_josh3.py:709: RuntimeWarning: overflow encountered in exp
  metrics['perplexity'] = np.exp(xentropy_avg)
/users/6/croal008/5527

micro_xentropy_avg: 653.9478149414062
micro_pos_xentropy_avg: 2545.560302734375
micro_auroc: 0.874030070437741
micro_auprc: 0.6621407755697623
micro_fscore: 0.6362780184592203
micro_precision: 0.5394442670073459
micro_recall: 0.7754820936639119
micro_threshold: 1.0
micro_accuracy: 0.9631976672161923
perchart_xentropy_avg: (656.87726, 79.25402, 541.1102, 789.16)
perchart_pos_xentropy_avg: (2533.0305, 1331.1086, 469.91898, 4556.6265)
perchart_auroc: (0.8773167508570112, 0.04588137950919598, 0.8216322003959462, 0.9631218313007842)
perchart_auprc: (0.6665822432505273, 0.07783690250221965, 0.5300406024914274, 0.7965970819404691)
perchart_fscore: (0.628435166949842, 0.08877373083790888, 0.48048568950563747, 0.7856209150326796)
perchart_precision: (0.5418469262863777, 0.1332440408860179, 0.3718120805369127, 0.830110497237569)
perchart_recall: (0.7820393346663974, 0.09383834655749143, 0.6750418760469011, 0.9574468085106383)
perchart_threshold: (0.87404305, 0.33035094, 4.2062293e-05, 1.0)
perch